In [1]:
import pandas as pd
import findspark
findspark.init()
from typing import Iterator
import numpy as np
from collections import Counter
from string import digits 
from enum import Enum
from typing import List, Optional
import re
import emojis
import emoji
import emot
import json
from redditscore.tokenizer import CrazyTokenizer
import preprocessor as tp
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer, MWETokenizer, word_tokenize
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, strip_short, split_alphanum
import os
import pyspark
from pyspark.sql import SparkSession, Row, Window
from pyspark.sql.functions import udf, col, array, when, size, spark_partition_id, pandas_udf, PandasUDFType
import pyspark.sql.functions as func
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, ArrayType, FloatType, DoubleType, LongType, StructType, StructField, IntegerType, TimestampType, BooleanType, DecimalType
from urllib3.exceptions import ProtocolError
import sqlalchemy
from sqlalchemy.dialects import postgresql 
import psutil
from flashtext import KeywordProcessor
from itertools import chain
from textblob import TextBlob
from decimal import Decimal
import functools
import operator
import string 
import time
import urlexpander
from bs4 import BeautifulSoup
import base64
import requests 
import demoji

topic = "session_frame"

In [2]:
spark = SparkSession.builder \
    .appName("SoDa-TAP") \
    .master("local[*]") \
    .config("spark.local.dir", "/home/jovyan/sodatap") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config('spark.sql.repl.eagerEval.enabled', True) \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0") \
    .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer') \
    .config('spark.streaming.kafka.consumer.cache.enabled', 'false') \
    .config('spark.kryoserializer.buffer.max', '2000M') \
    .config('spark.driver.maxResultSize', '1G') \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cb4a8feb-c19c-4f78-8ccf-cace1223ef70;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central

In [3]:
sc = spark.sparkContext

In [4]:
def write_sink(df, epoch_id):
    socialData_df = df.toPandas()
    socialData_df.to_sql(topic+"_processed", 'crate://142.244.110.140:8004', if_exists='append', index=False, chunksize=100)

In [5]:
def infer_topic_schema_json(topic):
    df_json = (spark.read.format("kafka") \
                .option("kafka.bootstrap.servers", "142.244.110.140:9092") \
                .option("subscribe", topic) \
                .option("startingOffsets", "earliest") \
                .option("maxOffsetsPerTrigger", "1") \
                .option("failOnDataLoss", "false") \
                .load() \
                .withColumn("value", F.expr("string(value)")) \
                .select("value"))
    
    df_read = spark.read.json(df_json.rdd.map(lambda x: x.value), multiLine=True)
    return df_read.schema.json()

In [6]:
infer_schema = True
schema_location = "test_schema.json"

if not infer_schema: 
    try:
        with open(schema_location, 'r') as f:
            topic_schema_txt = json.load(f)
    except:
        infer_schema = True
        pass

if infer_schema:
    topic_schema_txt = infer_topic_schema_json(topic)
    with open(schema_location, 'w') as f:
        json.dump(topic_schema_txt, f)
        
topic_schema = StructType.fromJson(json.loads(topic_schema_txt))

22/06/28 21:35:11 WARN KafkaSourceProvider: maxOffsetsPerTrigger option ignored in batch queries
22/06/28 21:35:26 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/06/28 21:35:26 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/06/28 21:35:26 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/06/28 21:35:26 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
22/06/28 21:35:26 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted becau

In [9]:
json_results = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "142.244.110.140:9092") \
            .option("subscribe", topic) \
            .option("maxOffsetsPerTrigger", "100000") \
            .option("startingOffsets", "earliest") \
            .option("failOnDataLoss", "false") \
            .load() \
            .withColumn("value", F.expr("string(value)")) \
            .select("value") \
            .withColumn('value', F.from_json(col("value"), topic_schema)) \
            .select("value.*")


# csv_tweets = csv_tweets.repartition(64)

#debug_sink = json_results.writeStream \
#    .outputMode("update") \
#    .trigger(processingTime='1 seconds') \
#    .option("truncate", "false")\
#    .format("console") \
#    .start()

debug_sink = json_results.writeStream \
    .foreachBatch(write_sink) \
    .start()

debug_sink.awaitTermination()

22/06/28 21:37:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1a4ce0c0-3adb-463a-954d-1749f2a55fc0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/28 21:37:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/06/28 21:37:44 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 1932 milliseconds
22/06/28 21:37:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 1163 milliseconds
22/06/28 21:37:51 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000 milliseconds, but spent 1382 milliseconds
22/06/28 21:37:52 